In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np 
import scipy.io as sio
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
sns.set_theme()

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.set_default_dtype(torch.float32)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [3]:
data = sio.loadmat("./Dataset/comp_matrix.mat")["training_matrix"]

X = data[:, :-1].astype(np.float32)
y = data[:, -1].reshape(-1, 1).astype(np.float32)

# Inputs & Targets
X = torch.from_numpy(X).to(device)
y = torch.from_numpy(y).to(device)

X.shape, y.shape

(torch.Size([27326, 2]), torch.Size([27326, 1]))

In [4]:
# Expressive GRU
class ProGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ProGRU, self).__init__()
        self.lstm = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=2, bidirectional=True, batch_first=True)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        self.activation = nn.PReLU()
    
    def forward(self, x):
        batch_size = x.shape[0]
        out, hn = self.lstm(x.view(batch_size, 1, -1))
        out = self.activation(self.linear1(hn[0])) # self.linear1(out)
        out = self.activation(self.linear2(out))
        out = self.linear3(out)
        return out

In [21]:
 # GRU Inputs
input_size = 2
hidden_size = 256
output_size = 1

# FNN
model = ProGRU(input_size, hidden_size, output_size).to(device)
optimizer = optim.AdamW(model.parameters(), lr = 1e-4)

# Initial Velocity
v = torch.zeros_like(y, requires_grad=True).to(device)
delta_t = 0.05

# Loss Function
loss_fn = nn.MSELoss()

for i in range(X.shape[0]+1):
    with torch.no_grad(): 
        v[i] = v[i-1] + delta_t * model(X[i-1].reshape(-1, 2))
    
    if i % 4 == 0:
        loss = loss_fn(v[i], y[i].reshape(-1, 1))
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        
        with torch.no_grad():
            print(f"Iter: {i}, Train Loss: {loss:.3f}")

# print(v_pred)

Iter: 0, Train Loss: 0.000
Iter: 4, Train Loss: 0.000
Iter: 8, Train Loss: 0.000
Iter: 12, Train Loss: 0.000
Iter: 16, Train Loss: 0.000
Iter: 20, Train Loss: 0.000
Iter: 24, Train Loss: 0.000
Iter: 28, Train Loss: 0.000
Iter: 32, Train Loss: 0.000
Iter: 36, Train Loss: 0.000
Iter: 40, Train Loss: 0.000
Iter: 44, Train Loss: 0.000
Iter: 48, Train Loss: 0.001
Iter: 52, Train Loss: 0.001
Iter: 56, Train Loss: 0.001
Iter: 60, Train Loss: 0.001
Iter: 64, Train Loss: 0.001
Iter: 68, Train Loss: 0.001
Iter: 72, Train Loss: 0.001
Iter: 76, Train Loss: 0.001
Iter: 80, Train Loss: 0.002
Iter: 84, Train Loss: 0.002
Iter: 88, Train Loss: 0.002
Iter: 92, Train Loss: 0.002
Iter: 96, Train Loss: 0.002
Iter: 100, Train Loss: 0.002
Iter: 104, Train Loss: 0.003
Iter: 108, Train Loss: 0.003
Iter: 112, Train Loss: 0.003
Iter: 116, Train Loss: 0.003
Iter: 120, Train Loss: 0.003
Iter: 124, Train Loss: 0.016
Iter: 128, Train Loss: 0.004
Iter: 132, Train Loss: 0.004
Iter: 136, Train Loss: 0.004
Iter: 140, Tr

KeyboardInterrupt: 

In [7]:
len(v_pred)

10

In [6]:
y[0:10]

tensor([[0.],
        [nan],
        [nan],
        [nan],
        [0.],
        [nan],
        [nan],
        [nan],
        [0.],
        [nan]], device='cuda:0')